In [7]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [2]:
%%bash
export WD=~/Simulation_result
export CONF=$(pwd)/Exp_conf_test
export SCRIPT=$(pwd)/simulation.py

mkdir $WD
cd $WD

for CONFIG in $CONF/*.json
do
    python3 $SCRIPT $CONFIG
done

TensorFlow version: 1.10.0
TensorFlow version: 1.10.0
TensorFlow version: 1.10.0


2019-06-07 15:03:38.090797: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-06-07 15:03:45.465961: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-06-07 15:03:53.218328: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA


In [8]:
%%R
experiments <- list.dirs(path = "~/Simulation_result", full.names = TRUE, recursive = FALSE)

datalist <- c()
x <- c("Experiment", "Pattern", "Peak", "Path")

for (experiment in experiments){
    exp_name <- basename(experiment)
    patterns <- list.dirs(path = experiment, full.names = TRUE, recursive = FALSE)
    for (pattern in patterns) {
        pattern_name<-basename(pattern)
        peaks <- list.dirs(path = pattern, full.names = TRUE, recursive = FALSE)
        for (peak in peaks) {
            peak_name<-basename(peak)
            peak_file <- list.files(path = peak, full.names = TRUE, recursive = FALSE)[1]
            datalist <- rbind(datalist, c(exp_name,pattern_name,peak_name,peak_file))
        }
    }
}

datadf = data.frame(datalist)
colnames(datadf) <- x

In [9]:
%%bash
mkdir out

mkdir: out: File exists
mkdir: out: File exists


In [10]:
%%R
library(MMDiff3)
library(reshape2)
library(ggplot2)
library(GenomicRanges)
library(DiffBind)
library(devtools)

In [24]:
%%R
replicas <- as.list(unique(datadf[['Experiment']]))
n_mods <- 10

modnames<-sapply(0:9, function(i) paste0('mod_',i))
samplesheet <- data.frame(list(modnames, modnames, rep(1, 10), rep('Ctr', 10), rep(1,10), rep('macs', 10)))
colnames(samplesheet)<-c('SampleID', 'Tissue', 'Factor', 'Condition', 'Replicate', 'PeakCaller')

write.csv(samplesheet,'out/SampleSheet.csv', row.names=FALSE)
for (replica in replicas){
    replica_df <- subset(datadf, Experiment == replica)
    peaks <- as.list(unique(replica_df[['Peak']]))
    patterns <- as.list(unique(replica_df[['Pattern']]))
    
    for (pattern in patterns) { 
        ExperimentData <- list(genome='none',
                       dataDir=".",
                       sampleSheet ="out/SampleSheet.csv")
        MetaData <- list('ExpData' = ExperimentData)
        MMD <- DBAmmd(MetaData)

        peak_df <- data.frame(chr=c(0:(length(peaks)-1)))
        peak_df$start <- 0
        peak_df$end <- 3300
        regions <- makeGRangesFromDataFrame(peak_df)
        MMD <- setRegions(MMD, regions)
    
        Meta <- metaData(MMD)
        Meta$AnaData$pairedEnd <- rep(FALSE, n_mods)
        Meta$AnaData$PeakBoundary <- 200
        MMD@MetaData <- Meta
        ################################   
        pattern_df = subset(replica_df, Pattern == pattern)
        dir.create(paste0("./out","/",replica))
        dirname <- paste0("./out","/",replica,"/",pattern)
        dir.create(dirname)
                
        read_data = by(data = pattern_df$Path,
                     INDICES=pattern_df$Peak,
                     FUN=function(x) data.frame(read.csv(file=paste(x), header=TRUE, sep=",")))
    
        readList <- list()
        rawCounts = matrix(, nrow = length(peaks), ncol = n_mods)

        rnames <-c()
        cnames <-c()
          
        for (i in c(0:(length(peaks)-1))){
            rnames <-c(rnames, paste("chr",i, ":0-3300",sep=""))
        }
                       
        for(i in c(0:(n_mods-1))) {
            cnames <-c(cnames, paste("mod_",i, sep=""))
        }
                       
        rownames(rawCounts) <- rnames
        colnames(rawCounts) <- cnames
                       
        for(i in c(0:(n_mods-1))) {
            col <- paste("mod_",i, sep="")
            tempList = list() 
            j<-0
            for(peak in peaks){
                row<-paste("chr",j, ":0-3300",sep="")
                df<-subset(read_data[[peak]], modification==i)
                tempList[[row]]<-(as.numeric(df$position)+200+1)
                rawCounts[row, col] <- nrow(df)
                j<-j+1
            }
            readList[[col]] <- tempList
        }
                       
        MMD@Reads <- list('Center'=readList)
        MMD@RawTotalCounts = rawCounts
        MMD <- compDists(MMD,dist.method='MMD2')
        dists <-MMD@DISTs$MMD2
        write.csv(dists, file = paste0(dirname, "/dists.csv"))
    }
}

[1] "./out/Experiment One/pattern_0"
[1] 10
[1] 4
[1] 591.1413
[1] "./out/Experiment One/pattern_1"
[1] 10
[1] 4
[1] 619.4255
[1] "./out/Experiment One/pattern_2"
[1] 10
[1] 4
[1] 609.1725
[1] "./out/Experiment Three/pattern_0"
[1] 10
[1] 4
[1] 616.9507
[1] "./out/Experiment Three/pattern_1"
[1] 10
[1] 4
[1] 607.4047
[1] "./out/Experiment Three/pattern_2"
[1] 10
[1] 4
[1] 634.9819
[1] "./out/Experiment Two/pattern_0"
[1] 10
[1] 4
[1] 596.975
[1] "./out/Experiment Two/pattern_1"
[1] 10
[1] 4
[1] 628.0876
[1] "./out/Experiment Two/pattern_2"
[1] 10
[1] 4
[1] 614.4758


[1] "./out/Experiment One/pattern_0"
[1] 10
[1] 4
[1] 591.1413
[1] "./out/Experiment One/pattern_1"
[1] 10
[1] 4
[1] 619.4255
[1] "./out/Experiment One/pattern_2"
[1] 10
[1] 4
[1] 609.1725
[1] "./out/Experiment Three/pattern_0"
[1] 10
[1] 4
[1] 616.9507
[1] "./out/Experiment Three/pattern_1"
[1] 10
[1] 4
[1] 607.4047
[1] "./out/Experiment Three/pattern_2"
[1] 10
[1] 4
[1] 634.9819
[1] "./out/Experiment Two/pattern_0"
[1] 10
[1] 4
[1] 596.975
[1] "./out/Experiment Two/pattern_1"
[1] 10
[1] 4
[1] 628.0876
[1] "./out/Experiment Two/pattern_2"
[1] 10
[1] 4
[1] 614.4758


In [12]:
%%R
print(length(peaks))

[1] 10


[1] 10
